In [1]:
import uuid
import pandas as pd
from tqdm import tqdm
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer, CrossEncoder
from src.retrieval.dense.utils import download_hf_models
from src.preprocessor.utils.dataset_level import read_pickle

/home/thiendc/projects/.thienenv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
download_hf_models('Tnt3o5/8k_vi_le_v2', 'minilm_l6')
encoder = SentenceTransformer(model_name_or_path = './src/retrieval/dense/embeddings/models/minilm_l6')

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

In [3]:
relevant_docs = read_pickle("data/processed/relevant_docs.pkl")
queries = read_pickle("data/processed/queries_v2.pkl")
corpus = read_pickle("data/processed/corpus_v2.pkl")

In [4]:

def prepare_training_dataset(queries: dict, corpus: dict, relevant_docs: dict):
    anchors = []
    positives = []
    
    # Sử dụng tqdm để theo dõi tiến trình của vòng lặp
    for query_id, docs in tqdm(relevant_docs.items(), desc='Processing queries'):
        for doc_id in docs:
            try:
                # Thử truy cập cả query và document
                anchor = queries[str(query_id)]
                positive = corpus[str(doc_id)]

                # Nếu không gặp lỗi, append vào danh sách
                anchors.append(anchor)
                positives.append(positive)

            except KeyError as e:
                # In ra thông báo lỗi và tiếp tục
                # print(f"Lỗi KeyError: {e} - Bỏ qua query_id: {query_id}, doc_id: {doc_id}")
                continue

    df = {
        "anchor": anchors,
        "positive": positives
    }
    
    return pd.DataFrame(df)

docs  = prepare_training_dataset(queries, corpus, relevant_docs)

Processing queries: 100%|██████████| 119456/119456 [00:00<00:00, 610656.10it/s]


In [8]:
docs = docs[:10]

In [11]:
query_string = "Người học ngành quản lý khai thác công trình thủy lợi trình độ cao đẳng phải có khả năng học tập và nâng cao trình độ như thế nào?"
qdrant = QdrantClient(":memory:")

qdrant.recreate_collection(
    collection_name = "legal_dense_retrieval",
    vectors_config = models.VectorParams(   
        size= encoder.get_sentence_embedding_dimension(),
        distance = models.Distance.COSINE
        )
    )


/tmp/ipykernel_679489/1500673605.py:4: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [13]:
qdrant.upload_points(
    collection_name = "legal_dense_retrieval",
    points = [
        models.PointStruct( id = str(uuid.uuid4()), 
                            vector = encoder.encode(row[1]['anchor']),
                            payload = {"anchor": row[1]['anchor'],
                                      "positive": row[1]['positive']}
                        )
                        for row in docs.iterrows()
    ],
)


In [17]:
hits = qdrant.search(
    collection_name = "legal_dense_retrieval",
    query_vector = encoder.encode(query_string).tolist(),
    limit = 3
)

In [18]:
hits

[ScoredPoint(id='51df0ce0-8281-4bc8-beb8-2fd406fe1d96', version=0, score=0.6472786437257718, payload={'anchor': 'Người học ngành quản_lý khai_thác công_trình thủy_lợi trình_độ cao_đẳng phải có khả_năng học_tập và nâng_cao trình_độ như thế_nào ?', 'positive': 'Khả_năng học_tập , nâng_cao trình_độ - Khối_lượng khối_lượng kiến_thức tối_thiểu , yêu_cầu về năng_lực mà người học phải đạt được sau khi tốt_nghiệp ngành , nghề Mộc xây_dựng và trang_trí nội_thất , trình_độ cao_đẳng có_thể tiếp_tục phát_triển ở các trình_độ cao hơn ; - Người học sau tốt_nghiệp có năng_lực tự học , tự cập_nhật những tiến_bộ khoa_học công_nghệ trong phạm_vi ngành , nghề để nâng_cao trình_độ hoặc học liên_thông lên trình_độ cao hơn trong cùng ngành , nghề hoặc trong nhóm ngành , nghề hoặc trong cùng lĩnh_vực đào { tạo . / . } Người học ngành Mộc xây_dựng và trang_trí nội_thất trình_độ cao_đẳng phải có khả_năng học_tập , nâng_cao trình_độ như thế sau : - Khối_lượng khối_lượng kiến_thức tối_thiểu , yêu_cầu về năng_lực